In [151]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
load_dotenv()
USER: admin
PASS: TFUKDovezz220!!
SERVIDOR: gfi-wg.555988031712.us-east-1.redshift-serverless.amazonaws.com:5439
DB: core_db

True

In [73]:
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)  

In [7]:
host = os.getenv("REDSHIFT_HOST")
port = os.getenv("REDSHIFT_PORT")
database = os.getenv("REDSHIFT_DB")
user = os.getenv("REDSHIFT_USER")
password = os.getenv("REDSHIFT_PASSWORD")

In [245]:
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    print("Connection Successful!")
except Exception as e:
    print(f"Error al conectar: {e}")
    conn = None

Connection Successful!


In [259]:
query = """
SELECT 
    t.id,
    t.subject,
    t.status,
    t.createddate,
    t.completeddatetime,
    t.priority,
    t.ownerid,
    u.firstname ||' '|| u.lastname AS employee,
    u.username AS employee_email,
    m.case_type_name__c AS case_type_t,
    m.live_associated__c AS lives,
    m.live_saved__c AS lives_saved,
    pa.firstname || ' ' || pa.lastname AS principal_attorney,
    m.attorney_notes__c AS attorney_notes,
    m.case_number__c AS case_number,
    m.name AS matter_name,
    m.case_type__c AS case_type,
    m.cm_case_sub_type__c AS case_subtype,
    m.previous_case_stage__c AS previous_stage,
    m.litify_pm__matter_stage_activity_formula__c AS active_stage,
    m.litify_pm__display_name__c AS display_name,
    m.litify_pm__open_date__c AS open_date,
    m.litify_pm__status__c AS status,
    m.case_manager_notes__c AS case_manager_notes,
    m.no_of_days__c AS number_of_days,
    m.full_translation__c AS full_translation
FROM 
    litify.task AS t
LEFT JOIN 
    litify.dim_users AS u
    ON t.ownerid = u.id 
LEFT JOIN 
    litify.matter AS m
    ON t.whatid = m.id
LEFT JOIN 
    litify.dim_users AS pa  
    ON m.litify_pm__principal_attorney__c = pa.id 
WHERE t.subject IN (

        'Full Translation', 
        'Full Translations', 
        'Full translation', 
        'FULL TRANSLATION', 
        'full Translations')
    
    AND m.litify_pm__matter_stage_activity_formula__c IN (
    
        'ASSEMBLING',
        'ASSEMBLY',
        'ATTORNEY REVIEW',
        'BIOMETRICS-CASE PENDING',
        'CASE PREP',
        'CASE REVIEW',
        'DOCS COLLECTED - TRANSLATIONS NEEDED',
        'INTERVIEW',
        'ON HOLD WAITING FOR FOIA',
        'ON HOLD-FBI MISSING',
        'ON HOLD-RECORDS MISSING',
        'ON-HOLD FBI MISSING',
        'TRANSLATIONS NEEDED',
        'TRANSLATIONS- NEEDED',
        'WORKING ON DOCS',
        'ASSEMBLY / ASSEMBLING',
        'ON HOLD WAITING FOR FOIA / FBI MISSING / RECORDS MISSING / FBI MISSING',
        'DOCS COLLECTED - TRANSLATIONS NEEDED / TRANSLATIONS NEEDED / TRANSLATIONS- NEEDED')

"""

In [261]:
df = pd.read_sql(query, conn)

C:\Users\alexey_abogadajulia\AppData\Local\Temp\ipykernel_1436\1168034203.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [263]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  294 non-null    object        
 1   subject             294 non-null    object        
 2   status              294 non-null    object        
 3   createddate         294 non-null    datetime64[ns]
 4   completeddatetime   262 non-null    datetime64[ns]
 5   priority            294 non-null    object        
 6   ownerid             294 non-null    object        
 7   employee            294 non-null    object        
 8   employee_email      294 non-null    object        
 9   case_type_t         276 non-null    object        
 10  lives               294 non-null    int64         
 11  lives_saved         294 non-null    int64         
 12  principal_attorney  269 non-null    object        
 13  attorney_notes      9 non-null      object        

In [267]:
df.sample(5)

,id,subject,status,createddate,completeddatetime,priority,ownerid,employee,employee_email,case_type_t,lives,lives_saved,principal_attorney,attorney_notes,case_number,matter_name,case_type,case_subtype,previous_stage,active_stage,display_name,open_date,status,case_manager_notes,number_of_days,full_translation
100,00TNv00000PLsogMAD,Full translation,Completed,2025-04-01 17:25:05,2025-04-08 21:34:28,Normal,005Nv000005equfIAA,Naomi Moreno,nmoreno@abogadajulia.com,T-VISA,2,1,Ruth Pineda,None,24-0812,MAT-24080353989,T-VISA,T-VISA LT DV,CW TEAM EDITS,ATTORNEY REVIEW,"A- T1) TINOCO TINOCO, Patricia T2) BALAM GARMA...",2024-08-02,Open,**ATTORNEY REVIEW\r\nPL- reassigned to Ruth P ...,256,True
181,00TNv00000PNjE9MAL,Full Translation,Completed,2025-04-02 00:43:43,2025-04-10 01:56:20,Normal,005Nv000005Gf2jIAC,Berenice Ruiz ☯️,bruiz@abogadajulia.com,None,2,1,Xinia Bermudez,None,24-11375,MAT-24112955261,T-VISA RE-FILE,None,DOCS COLLECTED - TRANSLATIONS NEEDED,ATTORNEY REVIEW,"A - SOTO GOMEZ, Yeon Gelacio (S)(IL) REFILE TE...",2024-11-29,Open,None,132,True
201,00TNv00000PfGUCMA3,Full Translation,Pending edits/update,2025-04-07 18:40:08,NaT,Priority,005Nv000004aJZhIAM,🐲 Josue Campechano🐉,jcampechano@abogadajulia.com,T-VISA,2,1,Xinia Bermudez,None,24-09301,MAT-24093054601,T-VISA,T-VISA LT STT,CW TEAM EDITS,ATTORNEY REVIEW,"A - CRUZ GARCIA, Olvin Omar (S) (Out of State ...",2024-09-30,Open,**Case Review \r\n\r\nCI- Assined to Ana Krla ...,191,True
13,00TNv00000PRcETMA1,Full Translation,Pending edits/update,2025-04-02 21:15:10,NaT,Normal,005Nv000005es21IAA,Paula Camacho,pcamacho@abogadajulia.com,T-VISA,3,1,Ana Amores,None,24-10243,MAT-24103154872,T-VISA,T-VISA LT,CW TEAM EDITS,ATTORNEY REVIEW,"A- GONZALEZ REYES, Enrique T1 ( Out of State- ...",2024-10-31,Open,**CASE REVIEW\r\nPRPR- JAYLEEN 02/15,161,True
154,00TNv00000PMJU1MAP,Full Translations,Completed,2025-04-01 19:53:00,2025-04-08 21:24:42,Normal,005Nv000005es21IAA,Paula Camacho,pcamacho@abogadajulia.com,T-VISA,2,1,Ana Amores,None,24-0726,MAT-24070353714,T-VISA,T-VISA STT LT,CW TEAM EDITS,ATTORNEY REVIEW,"A- T1) DIAZ LOPEZ, Daniel T2) MORGADO GUTIERRE...",2024-07-03,Open,**ATTORNEY REVIEW\r\nPL- assigned to Ana 3/24\...,286,True


In [277]:
backlog_count = []

for date in daily_backlog['date']:
    # Filtrar los casos que están pendientes en el día actual
    # Un caso está pendiente si tiene tareas de "Full Translation" que no han sido completadas hasta esa fecha
    cases_active_on_day = df[
        (df['createddate'] <= date) &  # Casos creados hasta la fecha actual
        ((df['completeddatetime'].isna()) | (df['completeddatetime'] > date))  # Tareas no completadas o completadas después de la fecha
        & (df['subject'].str.contains('Full Translation', case=False))  # Solo tareas con "Full Translation"
    ]
    
    # Identificar los casos únicos (matters) que tienen tareas pendientes de traducción
    active_cases = cases_active_on_day['whatid'].unique()  # 'whatid' se refiere al ID del caso (matter)
    
    # Filtrar los casos para contar solo los casos pendientes
    cases_pending = [case for case in active_cases if not df[df['whatid'] == case]['full_translation__c'].all()]
    
    # Contar cuántos casos están pendientes en el día actual
    backlog_count.append(len(cases_pending))

# Asignar la cuenta de casos pendientes al DataFrame de tendencias diarias
daily_backlog['cases_pending_count'] = backlog_count

# Crear nuevas columnas para agrupar por mes, año, día y semana
daily_backlog['year'] = daily_backlog['date'].dt.year
daily_backlog['month'] = daily_backlog['date'].dt.month
daily_backlog['day'] = daily_backlog['date'].dt.day
daily_backlog['week'] = daily_backlog['date'].dt.isocalendar().week

# Crear el gráfico de barras con Plotly
fig = px.bar(daily_backlog, x='date', y='cases_pending_count', title="Trend of Pending Translation Cases",
             labels={'date': 'Date', 'cases_pending_count': 'Pending Cases'},
             color='cases_pending_count',  # Color por el número de casos pendientes
             text='cases_pending_count',  # Mostrar el número de casos pendientes sobre cada barra
             opacity=0.6)  # Hacer las barras algo transparentes

# Agregar una línea para mostrar la tendencia
fig.add_scatter(x=daily_backlog['date'], y=daily_backlog['cases_pending_count'], mode='lines+markers', 
                name='Trend', line=dict(color='red', width=2))

# Añadir un slider de fecha para que el gráfico se pueda ajustar en el tiempo
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=True),
        type="date"
    ),
)

# Añadir un dropdown para filtros por rango de fechas: día, semana, mes, año
fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'args': [{'x': [daily_backlog['date']], 'y': [daily_backlog['cases_pending_count']]}, {'xaxis.type': 'date'}],
                    'label': 'Day',
                    'method': 'relayout'
                },
                {
                    'args': [{'x': [daily_backlog['month']], 'y': [daily_backlog['cases_pending_count']]}, {'xaxis.type': 'linear'}],
                    'label': 'Month',
                    'method': 'relayout'
                },
                {
                    'args': [{'x': [daily_backlog['week']], 'y': [daily_backlog['cases_pending_count']]}, {'xaxis.type': 'linear'}],
                    'label': 'Week',
                    'method': 'relayout'
                },
                {
                    'args': [{'x': [daily_backlog['year']], 'y': [daily_backlog['cases_pending_count']]}, {'xaxis.type': 'linear'}],
                    'label': 'Year',
                    'method': 'relayout'
                }
            ],
            'direction': 'down',
            'pad': {'r': 10, 't': 10},
            'showactive': True,
            'x': 0.17,
            'xanchor': 'left',
            'y': 1.15,
            'yanchor': 'top'
        }
    ]
)

# Ajustar el layout para evitar el solapamiento
fig.update_layout(
    width=1200,  # Ancho del gráfico
    height=800,  # Alto del gráfico
    margin=dict(r=100),  # Aumentar el margen derecho
    legend=dict(
        x=1.05,  # Mover la leyenda hacia la derecha
        y=1,
        traceorder="normal",
        orientation="v",  # Poner la leyenda en orientación vertical
        font=dict(size=10),
        title=None
    )
)

# Mostrar el gráfico interactivo
fig.show()

KeyError: 'whatid'